# Weighted Jaccard Similarity

In this notebook, we will use cuGraph to compute the Weighted Jaccard similarity and run analytics on our training dataset. 

*__This example assumes that the user has gone through the Jaccard notebook first__*

Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Last Edit: 03/08/2019

RAPIDS Versions: 0.6.0  


Test Hardware
* GP100 32G, CUDA 9,2


## Introduction
Weighted Jaccard is similar to the Jaccard Similarity but takes into account weights placed on the nodes of the graph.  


given:
The neighbors of a vertex, v, is defined as the set, U, of vertices connected by way of an edge to vertex v, or N(v) = {U} where v ∈V and ∀ u∈U ∃ edge(v,u)∈E.
and
wt(i) is the weight on vertex i
   
we can now define weight summing function as<br>
$WT(U) = \sum_{v \in U} {wt(v)}$

$WtJaccard(i, j) = \frac{WT(N(i) \cap N(j))}{WT(N(i) \cup N(j))}$


To compute the weighted Jaccard similarity between each pair of vertices connected by an edge in cuGraph use:
**nvJaccard_w(input_graph, vect_weights_ptr)**

Input
* input_graph: A cugraph.Graph object
* vect_weights_ptr: An array of vertex weights

Returns: 
* __df__: cudf.DataFrame with three columns:
    * df['source']: The source vertex id.
    * df['destination']: The destination vertex id.
    * df['jaccard_coeff']: The weighted jaccard coefficient computed between the source and destination vertex.
        
<br><br>   
    
## cuGraph 0.6 Notice 
cuGraph version 0.6 has some limitations:
* Only Int32 Vertex ID are supported
* Only float (FP32) edge data is supported
* Vertex numbering is assumed to start at zero

These limitations are being addressed and will be fixed future versions.  
These example notebooks will illustrate how to manipulate the data so that it comforms to the current limitations 

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)

This is a small graph which allows for easy visual inspection to validate results.  

In [1]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

## Read the data using cuDF

In [2]:
# Test file   
datafile='../data/networks/karate-data.csv'

# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()) )

In [3]:
# Adjust the vertex ID
gdf["src_0"] = gdf["src"] - 1
gdf["dst_0"] = gdf["dst"] - 1

In [4]:
# create a Graph 
G = cugraph.Graph()
G.add_edge_list(gdf["src_0"], gdf["dst_0"])

### Compute PageRank to use as the vertex weights

In [5]:
# Call Pagerank on the graph to get weights to use:
pr_df = cugraph.pagerank(G)

In [6]:
pr_df.head().to_pandas()

,vertex,pagerank
0,0,0.097255
1,1,0.053046
2,2,0.057388
3,3,0.035959
4,4,0.022008


### Now compute the Weighted Jaccard 

In [8]:
# Call weighted Jaccard using the Pagerank scores as weights:
df = cugraph.nvJaccard_w(G, pr_df['pagerank'])

Find the most similar pair of vertices - *adjust the vertex ID by adding 1 to match  illustration*

In [10]:
bestEdge = 0
for i in range(len(df)):
    if df['jaccard_coeff'][i] > df['jaccard_coeff'][bestEdge]:
        bestEdge = i
        
print("Vertices " + str(df['source'][bestEdge] +1) + 
      " and " + str(df['destination'][bestEdge] + 1) + 
      " are most similar with score: " + str(df['jaccard_coeff'][bestEdge]))

Vertices 1 and 2 are most similar with score: 1.0


___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___